#### Realizado por: 
#### Desiderio Nevado López
#### Laura Martín Olinero 
#### Daniel Quesada Pérez de San Roman

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null

In [2]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Consulta 2 Apartado 1').getOrCreate()

In [5]:
spark

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = spark.read.csv('/content/drive/MyDrive/iot_telemetry_data.csv', header=True, inferSchema=True)

In [8]:
!head /content/drive/MyDrive/iot_telemetry_data.csv

"ts","device","co","humidity","light","lpg","motion","smoke","temp"
"1.5945120943859746E9","b8:27:eb:bf:9d:51","0.004955938648391245","51.0","false","0.00765082227055719","false","0.02041127012241292","22.7"
"1.5945120947355676E9","00:0f:00:70:91:0a","0.0028400886071015706","76.0","false","0.005114383400977071","false","0.013274836704851536","19.700000762939453"
"1.5945120980735729E9","b8:27:eb:bf:9d:51","0.004976012340421658","50.9","false","0.007673227406398091","false","0.02047512557617824","22.6"
"1.594512099589146E9","1c:bf:ce:15:ec:4d","0.004403026829699689","76.80000305175781","true","0.007023337145877314","false","0.018628225377018803","27.0"
"1.594512101761235E9","b8:27:eb:bf:9d:51","0.004967363641908952","50.9","false","0.007663577282372411","false","0.020447620810233658","22.6"
"1.5945121044684107E9","1c:bf:ce:15:ec:4d","0.004391003954583357","77.9000015258789","true","0.007009458543138704","false","0.01858890754005078","27.0"
"1.5945121054488637E9","b8:27:eb:bf:9d:51","0.00

In [9]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [10]:
creditsSchema = StructType([
    StructField('ts', TimestampType()),
    StructField('device', StringType()),
    StructField('co', DoubleType()),
    StructField('humidity', DoubleType()),
    StructField('light', BooleanType()),
    StructField('lpg', DoubleType()),
    StructField('motion', BooleanType()),
    StructField('smoke', DoubleType()),
    StructField('temp', DoubleType()),
])

In [11]:
datos = (spark.read.option("header", "true")
           .option("quote", "\"")  # Tuning escape double quotes
           .option("escape", "\"") # Tuning escape double quotes
           .csv('/content/drive/MyDrive/iot_telemetry_data.csv', schema=creditsSchema)
          )

In [12]:
import pyspark
pyspark.__version__

'3.4.1'

In [13]:
datos.show(10)

+----+-----------------+--------------------+-----------------+-----+--------------------+------+--------------------+------------------+
|  ts|           device|                  co|         humidity|light|                 lpg|motion|               smoke|              temp|
+----+-----------------+--------------------+-----------------+-----+--------------------+------+--------------------+------------------+
|null|b8:27:eb:bf:9d:51|0.004955938648391245|             51.0|false| 0.00765082227055719| false| 0.02041127012241292|              22.7|
|null|00:0f:00:70:91:0a|0.002840088607101...|             76.0|false|0.005114383400977071| false|0.013274836704851536|19.700000762939453|
|null|b8:27:eb:bf:9d:51|0.004976012340421658|             50.9|false|0.007673227406398091| false| 0.02047512557617824|              22.6|
|null|1c:bf:ce:15:ec:4d|0.004403026829699689|76.80000305175781| true|0.007023337145877314| false|0.018628225377018803|              27.0|
|null|b8:27:eb:bf:9d:51|0.00496736

# APARTADO 1
## Consulta 2
### Define la columna ts como EventTime de Spark Structured Streaming. Define ventanas de 60 segundos de duración cada una, sin solape entre ellas. Para cada ventana, calcula el número de veces que se ha detectado luz y el número de veces que se ha detectado movimiento.

In [45]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, count, when, from_unixtime
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, DoubleType, BooleanType

# Create a SparkSession instance
spark = SparkSession.builder \
    .appName("Practica Spark") \
    .getOrCreate()

spark.sparkContext.setLogLevel('error')

creditsSchema = StructType([
    StructField('ts', FloatType()),
    StructField('device', StringType()),
    StructField('co', DoubleType()),
    StructField('humidity', DoubleType()),
    StructField('light', BooleanType()),
    StructField('lpg', DoubleType()),
    StructField('motion', BooleanType()),
    StructField('smoke', DoubleType()),
    StructField('temp', DoubleType()),
    StructField('file_name', StringType()),
])

# Read the data from the folder with CSV files
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(creditsSchema) \
    .load('/content/drive/MyDrive/Salida')

# Transform the 'ts' column from float (unix timestamp) to timestamp
df = df.withColumn('ts', from_unixtime(col('ts')).cast(TimestampType()))

# Perform window-based aggregation
windowed_df = df.groupBy(window('ts', '60 seconds')) \
    .agg(count(when(col('light'), True)).alias('Luz_Detectada'),
         count(when(col('motion'), True)).alias('Movimiento_Detectado')) \
    .select('window.*', 'Luz_Detectada', 'Movimiento_Detectado')

# Configure partitions to improve efficiency
spark.conf.set("spark.sql.shuffle.partitions", "10")
windowed_df.show()

# Write the DataFrame to CSV - Specify the output path
output_path = '/content/drive/MyDrive/OutputFolder'
windowed_df.write \
    .format('csv') \
    .option('header', True) \
    .mode('overwrite') \
    .save(output_path)

# Stop the SparkSession
spark.stop()


+-------------------+-------------------+-------------+--------------------+
|              start|                end|Luz_Detectada|Movimiento_Detectado|
+-------------------+-------------------+-------------+--------------------+
|2020-07-16 10:10:00|2020-07-16 10:11:00|           18|                   0|
|2020-07-16 10:12:00|2020-07-16 10:13:00|           22|                   0|
|2020-07-16 10:18:00|2020-07-16 10:19:00|           18|                   0|
|2020-07-16 10:27:00|2020-07-16 10:28:00|           24|                   0|
|2020-07-16 10:57:00|2020-07-16 10:58:00|           24|                   0|
|2020-07-16 11:12:00|2020-07-16 11:13:00|           24|                   0|
|2020-07-16 11:20:00|2020-07-16 11:21:00|           20|                   0|
|2020-07-16 11:22:00|2020-07-16 11:23:00|           21|                   0|
|2020-07-16 11:26:00|2020-07-16 11:27:00|           16|                   0|
|2020-07-16 11:29:00|2020-07-16 11:30:00|           19|                   0|

### Repite la operación, pero ahora considerando un watermark de 20 segundos para incluir los datos en el cálculo.

In [48]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, count, when, from_unixtime
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, DoubleType, BooleanType

# Create a SparkSession instance
spark = SparkSession.builder \
    .appName("Practica Spark") \
    .getOrCreate()

spark.sparkContext.setLogLevel('error')

creditsSchema = StructType([
    StructField('ts', FloatType()),
    StructField('device', StringType()),
    StructField('co', DoubleType()),
    StructField('humidity', DoubleType()),
    StructField('light', BooleanType()),
    StructField('lpg', DoubleType()),
    StructField('motion', BooleanType()),
    StructField('smoke', DoubleType()),
    StructField('temp', DoubleType()),
    StructField('file_name', StringType()),
])

# Read the data from the folder with CSV files
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(creditsSchema) \
    .load('/content/drive/MyDrive/Salida')

# Transform the 'ts' column from float (unix timestamp) to timestamp
df = df.withColumn('ts', from_unixtime(col('ts')).cast(TimestampType()))

# Perform window-based aggregation
windowed_df = df.groupBy(window('ts', '20 seconds')) \
    .agg(count(when(col('light'), True)).alias('Luz_Detectada'),
         count(when(col('motion'), True)).alias('Movimiento_Detectado')) \
    .select('window.*', 'Luz_Detectada', 'Movimiento_Detectado')

# Configure partitions to improve efficiency
spark.conf.set("spark.sql.shuffle.partitions", "10")
windowed_df.show()

# Write the DataFrame to CSV - Specify the output path
output_path = '/content/drive/MyDrive/OutputFolder'
windowed_df.write \
    .format('csv') \
    .option('header', True) \
    .mode('overwrite') \
    .save(output_path)

# Stop the SparkSession
spark.stop()

+-------------------+-------------------+-------------+--------------------+
|              start|                end|Luz_Detectada|Movimiento_Detectado|
+-------------------+-------------------+-------------+--------------------+
|2020-07-16 10:16:20|2020-07-16 10:16:40|           23|                   0|
|2020-07-16 11:09:40|2020-07-16 11:10:00|           21|                   0|
|2020-07-16 11:50:20|2020-07-16 11:50:40|           19|                   0|
|2020-07-17 07:00:00|2020-07-17 07:00:20|           21|                   0|
|2020-07-17 07:02:20|2020-07-17 07:02:40|           19|                   0|
|2020-07-17 07:47:00|2020-07-17 07:47:20|           18|                   0|
|2020-07-17 07:49:20|2020-07-17 07:49:40|           22|                   0|
|2020-07-17 08:34:00|2020-07-17 08:34:20|           20|                   0|
|2020-07-15 15:36:20|2020-07-15 15:36:40|           20|                   0|
|2020-07-15 15:40:40|2020-07-15 15:41:00|           24|                   0|

En resumen, el código leerá datos de los archivos CSV, realizará operaciones para transformar y agregar los datos en ventanas de tiempo de 60 y 20 segundos, nos muestra los resultados en la consola y guarda los resultados en archivos CSV en una carpeta especificada.